In [1]:
!pip install transformers[torch] datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import evaluate
import numpy as np
from transformers import pipeline
import torch

In [3]:
def read_dataset(ontopic_name: str, offtopic_name: str) -> Dataset:
    results = []
    with open(ontopic_name, "r") as f:
        for question in list(f):
            results.append({
                "label": 1,
                "text": question.strip()
            })

    with open(offtopic_name, "r") as f:
        for question in list(f):
            results.append({
                "label": 0,
                "text": question.strip()
            })

    return Dataset.from_pandas(pd.DataFrame(results))

In [4]:
model_name = "bert-base-cased"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [6]:
train_dataset = read_dataset("company-train.txt", "party-train.txt")
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)

test_dataset = read_dataset("company-test.txt", "party-test.txt")
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=3)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)

In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.034291,1.000000
2,No log,0.003409,1.000000
3,No log,0.002039,1.000000


TrainOutput(global_step=66, training_loss=0.10022073803525983, metrics={'train_runtime': 51.2865, 'train_samples_per_second': 10.003, 'train_steps_per_second': 1.287, 'total_flos': 134975971399680.0, 'train_loss': 0.10022073803525983, 'epoch': 3.0})

In [10]:
device = torch.device("cuda")
pipe = pipeline(model=model, task="text-classification", tokenizer=tokenizer, device=device)

In [13]:
pipe("When does the party happen?")

[{'label': 'LABEL_0', 'score': 0.9951661825180054}]

In [15]:
pipe("Who is PrismaLab's CEO?")

[{'label': 'LABEL_1', 'score': 0.9988982677459717}]

In [14]:
trainer.save_model()